# CulinaryFlo Datamart Integration with Everytable

By James, Curtis, and Richard begun November 2020

## Subscription Orders

In [5]:
-- SQL snippets from "ET-CF Test Data" documentation
-- https://docs.google.com/spreadsheets/d/1ooMnD9hrYZPGwEBwFJiyVWV1AE1206dYtyY_plA1B8E

SELECT 'Processing' AS 'Order Status'
       ,o.TotalAmount AS stripeFee
       ,CAST(o.Number AS nvarchar(99)) + '-' + CAST(p.Id AS nvarchar(99)) AS 'Order Number-Product Id'
       ,o.TotalAmount AS stripeNet
       ,o.CreatedDateUtc AS 'Order Date'
       ,o.Number AS 'Order Number'
       ,o.TotalAmount AS 'Order Total Amount'
       ,NULL AS Done -- Looks blank in Subs Orders
       ,o.CustomerId as 'Customer User ID'
       ,p.InternalName AS 'Item Name'
       ,i.TotalAmount AS 'Order Line Total (include tax)'
       ,i.OrderedQuantity AS 'Quantity (- Refund)' -- Or should it be (i.FulfilledQuantity - i.RefundedQuantity)
       ,p.Id AS 'Product Id'
       ,o.DeliveryZipCode AS 'Postcode (Shipping)'
       ,o.DeliveryFee AS 'Order Shipping Amount'
       ,IIF(s.Name IS NOT NULL, s.Name, dz.Description) AS SHOP -- Zone not flowing? (Field repeated below)
       ,prom.Name AS 'Coupon Code'
       ,NULL AS '#REF' -- Looks blank in Subs Orders
       ,o.DiscountAmount AS 'Discount Amount'
       ,'_tbd' AS 'Is Renewal?'
       ,o.FulfillmentDate AS dDate   -- Does it really needs conditions or is it fine?
       ,IIF(o.InStorePickupTime IS NOT NULL, CAST(o.InStorePickupTime AS nvarchar(99)),
            CAST(adzwi.StartHour AS nvarchar(9)) + ':00-' + CAST(adzwi.EndHour AS nvarchar(9)) + ':00') AS dTime
            -- Needs more conditions? InStorePickupTime is right? I could convert to AM/PM
            -- OnDemandDeliveryDateUtc for Delivery
       ,c.PhoneNumber AS Phone
       ,c.Email AS Email
       ,c.FirstName + ' ' + c.LastName AS CustomerName
       ,o.Instructions AS 'Customer Note'
       ,o.FulfillmentMethod AS Delivery_Type      -- NEED ENUMERATION TRANSLATION!!!!!!!!!!!!!!!!!!!!!
       ,'_tbd' as packaging_type                  -- SHOULD BE insulated_box, paper_bag !!!!!!!!!!!!!!
       ,o.StoreId AS Store_ID                     -- SHOULD BE home-delivery vs apartment-delivery !!!
       ,IIF(s.Name IS NOT NULL, s.Name, dz.Description) AS shipping_shop_pickup -- Zone not flowing? (Field repeated below)
       ,o.Instructions AS 'Order Note'
       ,NULL AS '#N/A'       -- Let Google Sheets do it for Phase I
       ,NULL AS 'a1, a2, a3' -- Let Google Sheets do it for Phase I
       ,c.FirstName AS custFirst
       ,c.LastName AS custLast
       ,NULL AS ItemType  -- Looks blank in Subs Orders
       ,NULL AS MealCount -- Looks blank in Subs Orders
       ,o.DeliveryAddress1 AS a1
       ,o.DeliveryAddress2 AS a2
       ,o.DeliveryCity + ', ' + /*CAST(o.DeliveryState AS nvarchar(99))*/ 'CA' + ' ' + o.DeliveryZipCode AS a3
                                -- Returns code 8, hardwired to 'CA' for now
        -- Not used
        --,o.DeliveryZoneWindowInstanceId
        --,dz.Description
        --,o.FulfillmentMethod
        --,subs.Instructions
  FROM ActiveOrders AS o
  LEFT JOIN ActiveOrderItems AS i ON (o.Id = i.OrderId)
  LEFT JOIN Products AS p ON (i.ProductId = p.Id)
  LEFT JOIN Stores AS s ON (o.StoreId = s.Id)
  LEFT JOIN Customers AS c ON (o.CustomerId = c.Id)
  --LEFT JOIN ProductCategories as pc ON (p.CategoryId = pc.Id)
  --LEFT JOIN Subscriptions AS subs ON (o.SubscriptionId = subs.Id)
  LEFT JOIN ActivePromotions AS prom ON (o.PromotionId = prom.Id)
  LEFT JOIN ActiveDeliveryZoneWindowInstances AS adzwi ON (o.DeliveryZoneWindowInstanceId = adzwi.Id)
  LEFT JOIN DeliveryZones AS dz ON (adzwi.DeliveryZoneWindowId = dz.Id)

(141 rows affected)

Total execution time: 00:00:00.154

Order Status,stripeFee,Order Number-Product Id,stripeNet,Order Date,Order Number,Order Total Amount,Done,Customer User ID,Item Name,Order Line Total (include tax),Quantity (- Refund),Product Id,Postcode (Shipping),Order Shipping Amount,SHOP,Coupon Code,Not Done,Discount Amount,Is Renewal?,dDate,dTime,Phone,Email,CustomerName,Customer Note,Delivery_Type,packaging_type,Store_ID,shipping_shop_pickup,Order Note,#N/A,"a1, a2, a3",custFirst,custLast,ItemType,MealCount,a1,a2,a3
Processing,46.74,196-EE08DA97-B9E5-478A-A42F-412EFF974E12,46.74,2020-11-17 04:16:38.9782614,196,46.74,_tbd,c1d9307c-66b4-417f-b49c-e7ebdd77f116,OATS WITH FRUIT & NUTS,8.95,1,ee08da97-b9e5-478a-a42f-412eff974e12,NULL,0.00,Compton,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,0,5129543328,meganmclindon+testCF2@gmail.com,Megan M.,NULL,1,_tbd,53058262-43c5-4a1d-bdc4-7801c862c1d0,Compton,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,NULL,NULL,NULL
Processing,46.74,196-D182AE0B-59AE-4403-BEA0-5517A56A6CD0,46.74,2020-11-17 04:16:38.9782614,196,46.74,_tbd,c1d9307c-66b4-417f-b49c-e7ebdd77f116,Golden Couscous Bowl,8.95,1,d182ae0b-59ae-4403-bea0-5517a56a6cd0,NULL,0.00,Compton,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,0,5129543328,meganmclindon+testCF2@gmail.com,Megan M.,NULL,1,_tbd,53058262-43c5-4a1d-bdc4-7801c862c1d0,Compton,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,NULL,NULL,NULL
Processing,46.74,196-94E4C3F4-F443-438E-9C7E-9C12C46D011E,46.74,2020-11-17 04:16:38.9782614,196,46.74,_tbd,c1d9307c-66b4-417f-b49c-e7ebdd77f116,Buffalo Chicken Wrap,8.95,1,94e4c3f4-f443-438e-9c7e-9c12c46d011e,NULL,0.00,Compton,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,0,5129543328,meganmclindon+testCF2@gmail.com,Megan M.,NULL,1,_tbd,53058262-43c5-4a1d-bdc4-7801c862c1d0,Compton,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,NULL,NULL,NULL
Processing,46.74,196-ADC7C6BE-B75B-47D8-94B4-E2DBE62F0F57,46.74,2020-11-17 04:16:38.9782614,196,46.74,_tbd,c1d9307c-66b4-417f-b49c-e7ebdd77f116,Turkey Sausage & Egg White Wrap,8.95,1,adc7c6be-b75b-47d8-94b4-e2dbe62f0f57,NULL,0.00,Compton,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,0,5129543328,meganmclindon+testCF2@gmail.com,Megan M.,NULL,1,_tbd,53058262-43c5-4a1d-bdc4-7801c862c1d0,Compton,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,NULL,NULL,NULL
Processing,46.74,196-7F12C304-40AB-4176-AC44-BF89431E64FF,46.74,2020-11-17 04:16:38.9782614,196,46.74,_tbd,c1d9307c-66b4-417f-b49c-e7ebdd77f116,Monica's Breakfast Burrito,8.95,1,7f12c304-40ab-4176-ac44-bf89431e64ff,NULL,0.00,Compton,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,0,5129543328,meganmclindon+testCF2@gmail.com,Megan M.,NULL,1,_tbd,53058262-43c5-4a1d-bdc4-7801c862c1d0,Compton,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,NULL,NULL,NULL
Processing,48.67,192-AC9C6CB7-52BA-4786-BF3C-D20D502FFB2A,48.67,2020-11-17 04:00:15.4769290,192,48.67,_tbd,0275cb7c-69a4-4e02-86f1-db1389cd509f,Lemon Pepper Chicken Caesar Salad,8.95,1,ac9c6cb7-52ba-4786-bf3c-d20d502ffb2a,90007,7.50,South LA/USC,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,NULL,5129543328,meganmclindon+testcf1@gmail.com,Megan M.,NULL,2,_tbd,605498ce-de42-4a95-98e4-8a2e37c4b155,South LA/USC,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,1101 W 23rd St,NULL,"Los Angeles, CA 90007"
Processing,48.67,192-BAD2DB12-9F96-412B-9F81-7B65770A0516,48.67,2020-11-17 04:00:15.4769290,192,48.67,_tbd,0275cb7c-69a4-4e02-86f1-db1389cd509f,BUFFALO CHICKEN SALAD,8.95,1,bad2db12-9f96-412b-9f81-7b65770a0516,90007,7.50,South LA/USC,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,NULL,5129543328,meganmclindon+testcf1@gmail.com,Megan M.,NULL,2,_tbd,605498ce-de42-4a95-98e4-8a2e37c4b155,South LA/USC,NULL,NULL,NULL,Megan,M.,_tbd,_tbd,1101 W 23rd St,NULL,"Los Angeles, CA 90007"
Processing,48.67,192-764A18E0-88DF-42A9-9F5C-A02975A77827,48.67,2020-11-17 04:00:15.4769290,192,48.67,_tbd,0275cb7c-69a4-4e02-86f1-db1389cd509f,Tangy BBQ Chicken Salad,8.95,1,764a18e0-88df-42a9-9f5c-a02975a77827,90007,7.50,South LA/USC,NULL,_tbd,0.00,_tbd,2020-11-19 00:00:00.0000000,NULL,5129543328,meganmclindon+testcf1@gmail.com,Megan M.,NULL,2,_tbd,605498c

## Store Orders

In [4]:
-- Store orders aka ActiveStoreProductInventoryRequests
SELECT o.Date
      ,s.Name as Store
      ,pc.Name as Category
      ,p.InternalName as Item
      ,SUM(o.ProductCount) as Quantity
  FROM ActiveStoreProductInventoryRequests AS o
  LEFT JOIN Products as p ON (o.ProductId = p.Id)
  LEFT JOIN Stores as s ON (o.StoreId = s.Id)
  LEFT JOIN ProductCategories as pc ON (p.CategoryId = pc.Id)
  GROUP BY o.date, s.Name, pc.Name, p.InternalName
  ORDER BY o.date, s.Name, pc.Name, p.InternalName

Warning: Null value is eliminated by an aggregate or other SET operation.

(1080 rows affected)

Total execution time: 00:00:00.194

Date,Store,Category,Item,Quantity
2020-11-16 00:00:00.0000000,Compton,Breakfast,Monica's Breakfast Burrito,10
2020-11-16 00:00:00.0000000,Compton,Breakfast,OATS WITH FRUIT & NUTS,6
2020-11-16 00:00:00.0000000,Compton,Breakfast,Scrambled Eggs & Veggies,NULL
2020-11-16 00:00:00.0000000,Compton,Breakfast,Southwest Breakfast Bowl,NULL
2020-11-16 00:00:00.0000000,Compton,Breakfast,Turkey Sausage & Egg White Wrap,NULL
2020-11-16 00:00:00.0000000,Compton,Hot Plates,Balsamic Chicken Grain Bowl,NULL
2020-11-16 00:00:00.0000000,Compton,Hot Plates,Butter Chicken,NULL
2020-11-16 00:00:00.0000000,Compton,Hot Plates,Carnitas Bowl,NULL
2020-11-16 00:00:00.0000000,Compton,Hot Plates,Chimichurri Chicken,NULL
2020-11-16 00:00:00.0000000,Compton,Hot Plates,Golden Couscous Bowl,NULL
